In [7]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import scipy.sparse as sp
from copy import deepcopy
import warnings 
import os
from sklearn.model_selection import KFold
import json
warnings.filterwarnings("ignore") 
import random
random.seed(1234)
np.random.seed(1234)

In [8]:
def obtain_data(task, isbalance, balance):
    
    pwd = './Data pre-processing result/balance/'
    if isbalance:
        node_feature_label = pd.read_csv(pwd + 'node_feature_label__balance.csv', index_col = 0)
    else:
        node_feature_label = pd.read_csv(pwd + 'node_feature_label__nobalance.csv', index_col = 0)
    
    train_test_id_idx = np.load('./Data pre-processing result/balance/task_' + task + balance + '__testlabel0_knn_edge_train_test_index_all.npz', allow_pickle = True)
    train_index_all = train_test_id_idx['train_index_all']##k=3的所有5flod数据放在一起
    test_index_all = train_test_id_idx['test_index_all']
    

    
    num_node = node_feature_label.shape[0]   ###节点总数3508
    node_feat = node_feature_label.iloc[:, 3:]   ####所有的节点特征3508x848
    label = node_feature_label['label']   ##所有的节点标签  3508
    
    print('num_node',num_node)
    print('node_feat',node_feat)
    print(label)

    mirna_ids = list(set(node_feature_label['Drug']))  ##184   184种药物对应的序号
    disease_ids = list(set(node_feature_label['Mutation']))###661  661种突变对应的序号
    random.shuffle(mirna_ids) #将序列的所有元素随机排序。
    random.shuffle(disease_ids)
    print('# Drug = {} | Mutation = {}'.format(len(mirna_ids), len(disease_ids)))
    
    #print('mirna_ids',mirna_ids)
    #print('disease_ids',disease_ids)
    
    
    mirna_test_num = int(len(mirna_ids) / 5)
    disease_test_num = int(len(disease_ids) / 5)
    print('# Test: Drug = {} | Mutation = {}'.format(mirna_test_num, disease_test_num))
    ##从全部数据中选择1/5的测试集数据
    
    
    return node_feature_label, num_node, node_feat, label, mirna_ids, disease_ids, train_index_all, test_index_all

In [13]:
def generate_graphsaint_data(task, train_index_all, test_index_all, node_feat, n_neigh, label, num_node, balance):

    fold = 0
    for train_idx, test_idx in zip(train_index_all, test_index_all): #train_index与test_index为下标
        # read knn_graph
        pwd = './Data pre-processing result/balance/'
        knn_graph_file = 'task_' + task + balance + '__testlabel0_knn' + str(n_neigh) + 'neighbors_edge__fold' + str(fold) + '.npz'
        knn_neighbors_graph = sp.load_npz(pwd + knn_graph_file)

        edge_src_dst = knn_neighbors_graph.nonzero()
        #print(edge_src_dst)
        
        #save dir
        save_dir = './Data pre-processing result/balance/task_' + task + balance + '__testlabel0_' + str(n_neigh) + 'knn_edge_fold' + str(fold) + '/'

        try:
            os.mkdir(save_dir)
        except OSError as error:
            print(error, save_dir)    
        ##先执行try的语句   如果错误  就执行except的语句   程序继续向下执行    
            
            
            
        # feats.npy，不需要自己标准化！因为在utils.py中的load_data中有标准化的步骤哦！
        feats = np.array(node_feat)
        np.save(save_dir + 'feats.npy', feats)
        #print('feats',feats)  ##feats   节点特征3508x848
        
        try:
            train_idx, test_idx = train_idx.tolist(), test_idx.tolist()
        except:
            train_idx, test_idx = train_idx, test_idx
            
        # role.json
        role = dict()
        role['tr'] = train_idx   ##2806
        role['va'] = test_idx    ###702
        role['te'] = test_idx    ###702 
        with open(save_dir + 'role.json','w') as f:
            json.dump(role, f)
        #print('json.dump(role, f)',role)   
        #print('role',len(train_idx))

        # class_map.json
        y = np.array(label)
        #print('y',len(y))
        class_map = dict()
        for i in range(num_node):
            class_map[str(i)] = y[i].tolist()
        with open(save_dir + 'class_map.json', 'w') as f:
            json.dump(class_map, f)
        #print('class_map:',class_map)
        
            
        # adj_*.npz
        train_idx_set = set(train_idx)
        test_idx_set = set(test_idx)
        
        row_full, col_full = edge_src_dst[0], edge_src_dst[1]
        
        row_train = []
        col_train = []
        row_val = []
        col_val = []
        for i in tqdm(range(row_full.shape[0])):
            if row_full[i] in train_idx_set and col_full[i] in train_idx_set:
                row_train.append(row_full[i])
                col_train.append(col_full[i])
            if row_full[i] in test_idx_set and col_full[i] in test_idx_set:
                row_val.append(row_full[i])
                col_val.append(col_full[i])

        row_train = np.array(row_train)
        col_train = np.array(col_train)
        row_val = np.array(row_val)
        col_val = np.array(col_val)
        dtype = np.bool_
        #dtype = np.bool

        adj_full = sp.coo_matrix(
            (
                np.ones(row_full.shape[0], dtype=dtype),
                (row_full, col_full),
            ),
            shape=(num_node, num_node)
        ).tocsr()
        
        #print('adj_full',adj_full)
        

        adj_train = sp.coo_matrix(
            (
                np.ones(row_train.shape[0], dtype=dtype),
                (row_train, col_train),
            ),
            shape=(num_node, num_node)
        ).tocsr()
        
        
        #print('adj_train',adj_train)

        adj_val = sp.coo_matrix(
            (
                np.ones(row_val.shape[0], dtype=dtype),
                (row_val, col_val),
            ),
            shape=(num_node, num_node)
        ).tocsr()
        
        #print('adj_val',adj_val)

        #print('adj_full  num edges:', adj_full.nnz)
        #print('adj_val   num edges:', adj_val.nnz)
        #print('adj_train num edges:', adj_train.nnz)
        sp.save_npz(save_dir + 'adj_full.npz', adj_full)
        sp.save_npz(save_dir + 'adj_train.npz', adj_train)
        sp.save_npz(save_dir + 'adj_val.npz', adj_val) # adj_val not used in GraphSAINT source code

        fold += 1
    
    print('--Complete--', fold)
    return   feats, role, class_map, adj_full, adj_train, adj_val, edge_src_dst

In [14]:
def run(task, isbalance):
    
    if isbalance:
        balance = ''
    else:
        balance = '__nobalance'

    for n_neigh in [1 ,3, 5, 7, 10, 15]:
    #for n_neigh in [3, 5, 7, 10, 15]:

        node_feature_label, num_node, node_feat, label, mirna_ids, disease_ids, train_index_all, test_index_all = obtain_data(task, 
                                                                                                                              isbalance,
                                                                                                                              balance)
        feats, role, class_map, adj_full, adj_train, adj_val, edge_src_dst = generate_graphsaint_data(task, 
                                                                                        train_index_all, 
                                                                                        test_index_all, 
                                                                                        node_feat, 
                                                                                        n_neigh, 
                                                                                        label, 
                                                                                        num_node, 
                                                                                        balance)
 
    return node_feature_label, num_node, node_feat, label, mirna_ids, disease_ids, train_index_all, test_index_all, \
    feats, role, class_map, adj_full, adj_train, adj_val, edge_src_dst

# RUN balance

In [15]:
for task in ['Tp']:
    node_feature_label, num_node, node_feat, label, mirna_ids, disease_ids, train_index_all, test_index_all, \
    feats, role, class_map, adj_full, adj_train, adj_val, edge_src_dst = run(task = task, isbalance =True)

num_node 1670
node_feat            0_x       1_x       2_x       3_x       4_x       5_x       6_x  \
0     1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
1     0.897662  1.000000  0.973867  0.967525  0.998237  0.974889  0.990946   
2     0.969539  0.973867  1.000000  0.997122  0.973182  0.986712  0.994273   
3     0.895714  0.998237  0.973182  0.964158  1.000000  0.966592  0.990725   
4     0.895714  0.998237  0.973182  0.964158  1.000000  0.966592  0.990725   
...        ...       ...       ...       ...       ...       ...       ...   
1665  0.985151  0.875795  0.946596  0.963267  0.865546  0.956866  0.922686   
1666  0.995515  0.900413  0.968720  0.979297  0.894133  0.962970  0.945407   
1667  0.986313  0.955002  0.995024  0.995447  0.955632  0.979596  0.985605   
1668  0.969931  0.961463  0.982292  0.991723  0.953832  0.992798  0.980442   
1669  0.991606  0.903490  0.967242  0.979399  0.894986  0.968853  0.945288   

           7_x       8_x       9_x  ...

100%|██████████| 1336/1336 [00:00<00:00, 1338970.17it/s]


--Complete-- 5
num_node 1670
node_feat            0_x       1_x       2_x       3_x       4_x       5_x       6_x  \
0     1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
1     0.897662  1.000000  0.973867  0.967525  0.998237  0.974889  0.990946   
2     0.969539  0.973867  1.000000  0.997122  0.973182  0.986712  0.994273   
3     0.895714  0.998237  0.973182  0.964158  1.000000  0.966592  0.990725   
4     0.895714  0.998237  0.973182  0.964158  1.000000  0.966592  0.990725   
...        ...       ...       ...       ...       ...       ...       ...   
1665  0.985151  0.875795  0.946596  0.963267  0.865546  0.956866  0.922686   
1666  0.995515  0.900413  0.968720  0.979297  0.894133  0.962970  0.945407   
1667  0.986313  0.955002  0.995024  0.995447  0.955632  0.979596  0.985605   
1668  0.969931  0.961463  0.982292  0.991723  0.953832  0.992798  0.980442   
1669  0.991606  0.903490  0.967242  0.979399  0.894986  0.968853  0.945288   

           7_x       8_x

100%|██████████| 4008/4008 [00:00<00:00, 2010617.20it/s]


--Complete-- 5
num_node 1670
node_feat            0_x       1_x       2_x       3_x       4_x       5_x       6_x  \
0     1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
1     0.897662  1.000000  0.973867  0.967525  0.998237  0.974889  0.990946   
2     0.969539  0.973867  1.000000  0.997122  0.973182  0.986712  0.994273   
3     0.895714  0.998237  0.973182  0.964158  1.000000  0.966592  0.990725   
4     0.895714  0.998237  0.973182  0.964158  1.000000  0.966592  0.990725   
...        ...       ...       ...       ...       ...       ...       ...   
1665  0.985151  0.875795  0.946596  0.963267  0.865546  0.956866  0.922686   
1666  0.995515  0.900413  0.968720  0.979297  0.894133  0.962970  0.945407   
1667  0.986313  0.955002  0.995024  0.995447  0.955632  0.979596  0.985605   
1668  0.969931  0.961463  0.982292  0.991723  0.953832  0.992798  0.980442   
1669  0.991606  0.903490  0.967242  0.979399  0.894986  0.968853  0.945288   

           7_x       8_x

...

--Complete-- 5
num_node 1670
node_feat            0_x       1_x       2_x       3_x       4_x       5_x       6_x  \
0     1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
1     0.897662  1.000000  0.973867  0.967525  0.998237  0.974889  0.990946   
2     0.969539  0.973867  1.000000  0.997122  0.973182  0.986712  0.994273   
3     0.895714  0.998237  0.973182  0.964158  1.000000  0.966592  0.990725   
4     0.895714  0.998237  0.973182  0.964158  1.000000  0.966592  0.990725   
...        ...       ...       ...       ...       ...       ...       ...   
1665  0.985151  0.875795  0.946596  0.963267  0.865546  0.956866  0.922686   
1666  0.995515  0.900413  0.968720  0.979297  0.894133  0.962970  0.945407   
1667  0.986313  0.955002  0.995024  0.995447  0.955632  0.979596  0.985605   
1668  0.969931  0.961463  0.982292  0.991723  0.953832  0.992798  0.980442   
1669  0.991606  0.903490  0.967242  0.979399  0.894986  0.968853  0.945288   

           7_x       8_x

100%|██████████| 13360/13360 [00:00<00:00, 3351229.08it/s]


--Complete-- 5
num_node 1670
node_feat            0_x       1_x       2_x       3_x       4_x       5_x       6_x  \
0     1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
1     0.897662  1.000000  0.973867  0.967525  0.998237  0.974889  0.990946   
2     0.969539  0.973867  1.000000  0.997122  0.973182  0.986712  0.994273   
3     0.895714  0.998237  0.973182  0.964158  1.000000  0.966592  0.990725   
4     0.895714  0.998237  0.973182  0.964158  1.000000  0.966592  0.990725   
...        ...       ...       ...       ...       ...       ...       ...   
1665  0.985151  0.875795  0.946596  0.963267  0.865546  0.956866  0.922686   
1666  0.995515  0.900413  0.968720  0.979297  0.894133  0.962970  0.945407   
1667  0.986313  0.955002  0.995024  0.995447  0.955632  0.979596  0.985605   
1668  0.969931  0.961463  0.982292  0.991723  0.953832  0.992798  0.980442   
1669  0.991606  0.903490  0.967242  0.979399  0.894986  0.968853  0.945288   

           7_x       8_x

100%|██████████| 20040/20040 [00:00<00:00, 3351295.89it/s]

--Complete-- 5


In [23]:
for task in ['Td']:
    node_feature_label, num_node, node_feat, label, mirna_ids, disease_ids, train_index_all, test_index_all, \
    feats, role, class_map, adj_full, adj_train, adj_val, edge_src_dst = run(task = task, isbalance = True)

num_node 1670
node_feat            0_x       1_x       2_x       3_x       4_x       5_x       6_x  \
0     1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
1     0.897662  1.000000  0.973867  0.967525  0.998237  0.974889  0.990946   
2     0.969539  0.973867  1.000000  0.997122  0.973182  0.986712  0.994273   
3     0.895714  0.998237  0.973182  0.964158  1.000000  0.966592  0.990725   
4     0.895714  0.998237  0.973182  0.964158  1.000000  0.966592  0.990725   
...        ...       ...       ...       ...       ...       ...       ...   
1665  0.985151  0.875795  0.946596  0.963267  0.865546  0.956866  0.922686   
1666  0.995515  0.900413  0.968720  0.979297  0.894133  0.962970  0.945407   
1667  0.986313  0.955002  0.995024  0.995447  0.955632  0.979596  0.985605   
1668  0.969931  0.961463  0.982292  0.991723  0.953832  0.992798  0.980442   
1669  0.991606  0.903490  0.967242  0.979399  0.894986  0.968853  0.945288   

           7_x       8_x       9_x  ...

100%|██████████| 1290/1290 [00:00<?, ?it/s]

--Complete-- 5


num_node 1670
node_feat            0_x       1_x       2_x       3_x       4_x       5_x       6_x  \
0     1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
1     0.897662  1.000000  0.973867  0.967525  0.998237  0.974889  0.990946   
2     0.969539  0.973867  1.000000  0.997122  0.973182  0.986712  0.994273   
3     0.895714  0.998237  0.973182  0.964158  1.000000  0.966592  0.990725   
4     0.895714  0.998237  0.973182  0.964158  1.000000  0.966592  0.990725   
...        ...       ...       ...       ...       ...       ...       ...   
1665  0.985151  0.875795  0.946596  0.963267  0.865546  0.956866  0.922686   
1666  0.995515  0.900413  0.968720  0.979297  0.894133  0.962970  0.945407   
1667  0.986313  0.955002  0.995024  0.995447  0.955632  0.979596  0.985605   
1668  0.969931  0.961463  0.982292  0.991723  0.953832  0.992798  0.980442   
1669  0.991606  0.903490  0.967242  0.979399  0.894986  0.968853  0.945288   

           7_x       8_x       9_x  ...

100%|██████████| 3870/3870 [00:00<00:00, 1941621.59it/s]

...

100%|██████████| 12900/12900 [00:00<00:00, 2874031.74it/s]

--Complete-- 5


num_node 1670
node_feat            0_x       1_x       2_x       3_x       4_x       5_x       6_x  \
0     1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
1     0.897662  1.000000  0.973867  0.967525  0.998237  0.974889  0.990946   
2     0.969539  0.973867  1.000000  0.997122  0.973182  0.986712  0.994273   
3     0.895714  0.998237  0.973182  0.964158  1.000000  0.966592  0.990725   
4     0.895714  0.998237  0.973182  0.964158  1.000000  0.966592  0.990725   
...        ...       ...       ...       ...       ...       ...       ...   
1665  0.985151  0.875795  0.946596  0.963267  0.865546  0.956866  0.922686   
1666  0.995515  0.900413  0.968720  0.979297  0.894133  0.962970  0.945407   
1667  0.986313  0.955002  0.995024  0.995447  0.955632  0.979596  0.985605   
1668  0.969931  0.961463  0.982292  0.991723  0.953832  0.992798  0.980442   
1669  0.991606  0.903490  0.967242  0.979399  0.894986  0.968853  0.945288   

           7_x       8_x       9_x  ...

100%|██████████| 19350/19350 [00:00<00:00, 2667711.35it/s]

--Complete-- 5


In [24]:
for task in ['Tm']:
    node_feature_label, num_node, node_feat, label, mirna_ids, disease_ids, train_index_all, test_index_all, \
    feats, role, class_map, adj_full, adj_train, adj_val, edge_src_dst = run(task = task, isbalance = True)

num_node 1670
node_feat            0_x       1_x       2_x       3_x       4_x       5_x       6_x  \
0     1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
1     0.897662  1.000000  0.973867  0.967525  0.998237  0.974889  0.990946   
2     0.969539  0.973867  1.000000  0.997122  0.973182  0.986712  0.994273   
3     0.895714  0.998237  0.973182  0.964158  1.000000  0.966592  0.990725   
4     0.895714  0.998237  0.973182  0.964158  1.000000  0.966592  0.990725   
...        ...       ...       ...       ...       ...       ...       ...   
1665  0.985151  0.875795  0.946596  0.963267  0.865546  0.956866  0.922686   
1666  0.995515  0.900413  0.968720  0.979297  0.894133  0.962970  0.945407   
1667  0.986313  0.955002  0.995024  0.995447  0.955632  0.979596  0.985605   
1668  0.969931  0.961463  0.982292  0.991723  0.953832  0.992798  0.980442   
1669  0.991606  0.903490  0.967242  0.979399  0.894986  0.968853  0.945288   

           7_x       8_x       9_x  ...

100%|██████████| 1254/1254 [00:00<?, ?it/s]


--Complete-- 5
num_node 1670
node_feat            0_x       1_x       2_x       3_x       4_x       5_x       6_x  \
0     1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
1     0.897662  1.000000  0.973867  0.967525  0.998237  0.974889  0.990946   
2     0.969539  0.973867  1.000000  0.997122  0.973182  0.986712  0.994273   
3     0.895714  0.998237  0.973182  0.964158  1.000000  0.966592  0.990725   
4     0.895714  0.998237  0.973182  0.964158  1.000000  0.966592  0.990725   
...        ...       ...       ...       ...       ...       ...       ...   
1665  0.985151  0.875795  0.946596  0.963267  0.865546  0.956866  0.922686   
1666  0.995515  0.900413  0.968720  0.979297  0.894133  0.962970  0.945407   
1667  0.986313  0.955002  0.995024  0.995447  0.955632  0.979596  0.985605   
1668  0.969931  0.961463  0.982292  0.991723  0.953832  0.992798  0.980442   
1669  0.991606  0.903490  0.967242  0.979399  0.894986  0.968853  0.945288   

           7_x       8_x

100%|██████████| 3762/3762 [00:00<00:00, 3773970.74it/s]


--Complete-- 5
num_node 1670
node_feat            0_x       1_x       2_x       3_x       4_x       5_x       6_x  \
0     1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
1     0.897662  1.000000  0.973867  0.967525  0.998237  0.974889  0.990946   
2     0.969539  0.973867  1.000000  0.997122  0.973182  0.986712  0.994273   
3     0.895714  0.998237  0.973182  0.964158  1.000000  0.966592  0.990725   
4     0.895714  0.998237  0.973182  0.964158  1.000000  0.966592  0.990725   
...        ...       ...       ...       ...       ...       ...       ...   
1665  0.985151  0.875795  0.946596  0.963267  0.865546  0.956866  0.922686   
1666  0.995515  0.900413  0.968720  0.979297  0.894133  0.962970  0.945407   
1667  0.986313  0.955002  0.995024  0.995447  0.955632  0.979596  0.985605   
1668  0.969931  0.961463  0.982292  0.991723  0.953832  0.992798  0.980442   
1669  0.991606  0.903490  0.967242  0.979399  0.894986  0.968853  0.945288   

           7_x       8_x

...

--Complete-- 5
num_node 1670
node_feat            0_x       1_x       2_x       3_x       4_x       5_x       6_x  \
0     1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
1     0.897662  1.000000  0.973867  0.967525  0.998237  0.974889  0.990946   
2     0.969539  0.973867  1.000000  0.997122  0.973182  0.986712  0.994273   
3     0.895714  0.998237  0.973182  0.964158  1.000000  0.966592  0.990725   
4     0.895714  0.998237  0.973182  0.964158  1.000000  0.966592  0.990725   
...        ...       ...       ...       ...       ...       ...       ...   
1665  0.985151  0.875795  0.946596  0.963267  0.865546  0.956866  0.922686   
1666  0.995515  0.900413  0.968720  0.979297  0.894133  0.962970  0.945407   
1667  0.986313  0.955002  0.995024  0.995447  0.955632  0.979596  0.985605   
1668  0.969931  0.961463  0.982292  0.991723  0.953832  0.992798  0.980442   
1669  0.991606  0.903490  0.967242  0.979399  0.894986  0.968853  0.945288   

           7_x       8_x

100%|██████████| 12540/12540 [00:00<00:00, 3144787.57it/s]


--Complete-- 5
num_node 1670
node_feat            0_x       1_x       2_x       3_x       4_x       5_x       6_x  \
0     1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
1     0.897662  1.000000  0.973867  0.967525  0.998237  0.974889  0.990946   
2     0.969539  0.973867  1.000000  0.997122  0.973182  0.986712  0.994273   
3     0.895714  0.998237  0.973182  0.964158  1.000000  0.966592  0.990725   
4     0.895714  0.998237  0.973182  0.964158  1.000000  0.966592  0.990725   
...        ...       ...       ...       ...       ...       ...       ...   
1665  0.985151  0.875795  0.946596  0.963267  0.865546  0.956866  0.922686   
1666  0.995515  0.900413  0.968720  0.979297  0.894133  0.962970  0.945407   
1667  0.986313  0.955002  0.995024  0.995447  0.955632  0.979596  0.985605   
1668  0.969931  0.961463  0.982292  0.991723  0.953832  0.992798  0.980442   
1669  0.991606  0.903490  0.967242  0.979399  0.894986  0.968853  0.945288   

           7_x       8_x

100%|██████████| 18810/18810 [00:00<00:00, 3145477.16it/s]

--Complete-- 5


# RUN nobalance

In [22]:
for task in ['Tp']:
    node_feature_label, num_node, node_feat, label, mirna_ids, disease_ids, train_index_all, test_index_all, \
    feats, role, class_map, adj_full, adj_train, adj_val, edge_src_dst = run(task = task, isbalance = False)
    


num_node 10524
node_feat             0_x       1_x       2_x       3_x       4_x       5_x       6_x  \
0      0.000000  0.227305  0.195368  0.222083  0.218782  0.199313  0.206095   
1      0.891151  0.959950  0.975833  0.987653  0.993496  0.967372  0.000000   
2      0.920408  0.950667  0.991906  0.995222  0.992450  0.991524  0.987794   
3      0.871441  0.889783  0.993341  0.968516  0.956858  0.998308  0.951931   
4      0.894709  0.969457  0.841373  0.915243  0.938411  0.825470  0.938192   
...         ...       ...       ...       ...       ...       ...       ...   
10519  0.785024  0.826036  0.983324  0.930918  0.917091  0.987983  0.920082   
10520  0.915948  0.968840  0.973972  0.995543  0.999113  0.967655  0.995371   
10521  0.948208  0.969684  0.975660  1.000000  0.997859  0.975191  0.984455   
10522  0.959047  0.993068  0.862791  0.944805  0.961731  0.853991  0.943685   
10523  0.973219  0.990284  0.883873  0.962145  0.971783  0.878055  0.945070   

            7_x       8_x 

100%|██████████| 31572/31572 [00:00<00:00, 791399.87it/s]


--Complete-- 5
num_node 10524
node_feat             0_x       1_x       2_x       3_x       4_x       5_x       6_x  \
0      0.000000  0.227305  0.195368  0.222083  0.218782  0.199313  0.206095   
1      0.891151  0.959950  0.975833  0.987653  0.993496  0.967372  0.000000   
2      0.920408  0.950667  0.991906  0.995222  0.992450  0.991524  0.987794   
3      0.871441  0.889783  0.993341  0.968516  0.956858  0.998308  0.951931   
4      0.894709  0.969457  0.841373  0.915243  0.938411  0.825470  0.938192   
...         ...       ...       ...       ...       ...       ...       ...   
10519  0.785024  0.826036  0.983324  0.930918  0.917091  0.987983  0.920082   
10520  0.915948  0.968840  0.973972  0.995543  0.999113  0.967655  0.995371   
10521  0.948208  0.969684  0.975660  1.000000  0.997859  0.975191  0.984455   
10522  0.959047  0.993068  0.862791  0.944805  0.961731  0.853991  0.943685   
10523  0.973219  0.990284  0.883873  0.962145  0.971783  0.878055  0.945070   

           

100%|██████████| 52620/52620 [00:00<00:00, 732786.42it/s]


--Complete-- 5
num_node 10524
node_feat             0_x       1_x       2_x       3_x       4_x       5_x       6_x  \
0      0.000000  0.227305  0.195368  0.222083  0.218782  0.199313  0.206095   
1      0.891151  0.959950  0.975833  0.987653  0.993496  0.967372  0.000000   
2      0.920408  0.950667  0.991906  0.995222  0.992450  0.991524  0.987794   
3      0.871441  0.889783  0.993341  0.968516  0.956858  0.998308  0.951931   
4      0.894709  0.969457  0.841373  0.915243  0.938411  0.825470  0.938192   
...         ...       ...       ...       ...       ...       ...       ...   
10519  0.785024  0.826036  0.983324  0.930918  0.917091  0.987983  0.920082   
10520  0.915948  0.968840  0.973972  0.995543  0.999113  0.967655  0.995371   
10521  0.948208  0.969684  0.975660  1.000000  0.997859  0.975191  0.984455   
10522  0.959047  0.993068  0.862791  0.944805  0.961731  0.853991  0.943685   
10523  0.973219  0.990284  0.883873  0.962145  0.971783  0.878055  0.945070   

           

...

--Complete-- 5
num_node 10524
node_feat             0_x       1_x       2_x       3_x       4_x       5_x       6_x  \
0      0.000000  0.227305  0.195368  0.222083  0.218782  0.199313  0.206095   
1      0.891151  0.959950  0.975833  0.987653  0.993496  0.967372  0.000000   
2      0.920408  0.950667  0.991906  0.995222  0.992450  0.991524  0.987794   
3      0.871441  0.889783  0.993341  0.968516  0.956858  0.998308  0.951931   
4      0.894709  0.969457  0.841373  0.915243  0.938411  0.825470  0.938192   
...         ...       ...       ...       ...       ...       ...       ...   
10519  0.785024  0.826036  0.983324  0.930918  0.917091  0.987983  0.920082   
10520  0.915948  0.968840  0.973972  0.995543  0.999113  0.967655  0.995371   
10521  0.948208  0.969684  0.975660  1.000000  0.997859  0.975191  0.984455   
10522  0.959047  0.993068  0.862791  0.944805  0.961731  0.853991  0.943685   
10523  0.973219  0.990284  0.883873  0.962145  0.971783  0.878055  0.945070   

           

100%|██████████| 105240/105240 [00:00<00:00, 620716.04it/s]


--Complete-- 5
num_node 10524
node_feat             0_x       1_x       2_x       3_x       4_x       5_x       6_x  \
0      0.000000  0.227305  0.195368  0.222083  0.218782  0.199313  0.206095   
1      0.891151  0.959950  0.975833  0.987653  0.993496  0.967372  0.000000   
2      0.920408  0.950667  0.991906  0.995222  0.992450  0.991524  0.987794   
3      0.871441  0.889783  0.993341  0.968516  0.956858  0.998308  0.951931   
4      0.894709  0.969457  0.841373  0.915243  0.938411  0.825470  0.938192   
...         ...       ...       ...       ...       ...       ...       ...   
10519  0.785024  0.826036  0.983324  0.930918  0.917091  0.987983  0.920082   
10520  0.915948  0.968840  0.973972  0.995543  0.999113  0.967655  0.995371   
10521  0.948208  0.969684  0.975660  1.000000  0.997859  0.975191  0.984455   
10522  0.959047  0.993068  0.862791  0.944805  0.961731  0.853991  0.943685   
10523  0.973219  0.990284  0.883873  0.962145  0.971783  0.878055  0.945070   

           

100%|██████████| 157860/157860 [00:00<00:00, 608778.93it/s]


--Complete-- 5


In [26]:
node_feature_label, num_node, node_feat, label, mirna_ids, disease_ids, train_index_all, test_index_all, \
feats, role, class_map, adj_full, adj_train, adj_val, edge_src_dst = run(task = 'Tm' , isbalance = False)

num_node 121624
node_feat              0_x       1_x       2_x       3_x       4_x       5_x       6_x  \
0       1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
1       1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
2       1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
3       1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
4       1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
...          ...       ...       ...       ...       ...       ...       ...   
121619  0.932862  0.981167  0.987583  0.978269  0.986323  0.962156  0.991868   
121620  0.932862  0.981167  0.987583  0.978269  0.986323  0.962156  0.991868   
121621  0.932862  0.981167  0.987583  0.978269  0.986323  0.962156  0.991868   
121622  0.932862  0.981167  0.987583  0.978269  0.986323  0.962156  0.991868   
121623  0.932862  0.981167  0.987583  0.978269  0.986323  0.962156  0.991868   

             

100%|██████████| 95184/95184 [00:00<00:00, 2867500.55it/s]


--Complete-- 5
num_node 121624
node_feat              0_x       1_x       2_x       3_x       4_x       5_x       6_x  \
0       1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
1       1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
2       1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
3       1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
4       1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
...          ...       ...       ...       ...       ...       ...       ...   
121619  0.932862  0.981167  0.987583  0.978269  0.986323  0.962156  0.991868   
121620  0.932862  0.981167  0.987583  0.978269  0.986323  0.962156  0.991868   
121621  0.932862  0.981167  0.987583  0.978269  0.986323  0.962156  0.991868   
121622  0.932862  0.981167  0.987583  0.978269  0.986323  0.962156  0.991868   
121623  0.932862  0.981167  0.987583  0.978269  0.986323  0.962156  0.991868   

100%|██████████| 285552/285552 [00:00<00:00, 2744217.78it/s]


--Complete-- 5
num_node 121624
node_feat              0_x       1_x       2_x       3_x       4_x       5_x       6_x  \
0       1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
1       1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
2       1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
3       1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
4       1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
...          ...       ...       ...       ...       ...       ...       ...   
121619  0.932862  0.981167  0.987583  0.978269  0.986323  0.962156  0.991868   
121620  0.932862  0.981167  0.987583  0.978269  0.986323  0.962156  0.991868   
121621  0.932862  0.981167  0.987583  0.978269  0.986323  0.962156  0.991868   
121622  0.932862  0.981167  0.987583  0.978269  0.986323  0.962156  0.991868   
121623  0.932862  0.981167  0.987583  0.978269  0.986323  0.962156  0.991868   

...

--Complete-- 5
num_node 121624
node_feat              0_x       1_x       2_x       3_x       4_x       5_x       6_x  \
0       1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
1       1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
2       1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
3       1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
4       1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
...          ...       ...       ...       ...       ...       ...       ...   
121619  0.932862  0.981167  0.987583  0.978269  0.986323  0.962156  0.991868   
121620  0.932862  0.981167  0.987583  0.978269  0.986323  0.962156  0.991868   
121621  0.932862  0.981167  0.987583  0.978269  0.986323  0.962156  0.991868   
121622  0.932862  0.981167  0.987583  0.978269  0.986323  0.962156  0.991868   
121623  0.932862  0.981167  0.987583  0.978269  0.986323  0.962156  0.991868   

100%|██████████| 951840/951840 [00:00<00:00, 2879942.81it/s]


--Complete-- 5
num_node 121624
node_feat              0_x       1_x       2_x       3_x       4_x       5_x       6_x  \
0       1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
1       1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
2       1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
3       1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
4       1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
...          ...       ...       ...       ...       ...       ...       ...   
121619  0.932862  0.981167  0.987583  0.978269  0.986323  0.962156  0.991868   
121620  0.932862  0.981167  0.987583  0.978269  0.986323  0.962156  0.991868   
121621  0.932862  0.981167  0.987583  0.978269  0.986323  0.962156  0.991868   
121622  0.932862  0.981167  0.987583  0.978269  0.986323  0.962156  0.991868   
121623  0.932862  0.981167  0.987583  0.978269  0.986323  0.962156  0.991868   

100%|██████████| 1427760/1427760 [00:00<00:00, 2787331.68it/s]


--Complete-- 5


In [27]:
# for n_neigh in [1, 3, 5, 7, 15]: 
node_feature_label, num_node, node_feat, label, mirna_ids, disease_ids, train_index_all, test_index_all, \
feats, role, class_map, adj_full, adj_train, adj_val, edge_src_dst = run(task = 'Td', isbalance = False)

num_node 121624
node_feat              0_x       1_x       2_x       3_x       4_x       5_x       6_x  \
0       1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
1       1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
2       1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
3       1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
4       1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
...          ...       ...       ...       ...       ...       ...       ...   
121619  0.932862  0.981167  0.987583  0.978269  0.986323  0.962156  0.991868   
121620  0.932862  0.981167  0.987583  0.978269  0.986323  0.962156  0.991868   
121621  0.932862  0.981167  0.987583  0.978269  0.986323  0.962156  0.991868   
121622  0.932862  0.981167  0.987583  0.978269  0.986323  0.962156  0.991868   
121623  0.932862  0.981167  0.987583  0.978269  0.986323  0.962156  0.991868   

             

100%|██████████| 97152/97152 [00:00<00:00, 2816943.92it/s]


--Complete-- 5
num_node 121624
node_feat              0_x       1_x       2_x       3_x       4_x       5_x       6_x  \
0       1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
1       1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
2       1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
3       1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
4       1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
...          ...       ...       ...       ...       ...       ...       ...   
121619  0.932862  0.981167  0.987583  0.978269  0.986323  0.962156  0.991868   
121620  0.932862  0.981167  0.987583  0.978269  0.986323  0.962156  0.991868   
121621  0.932862  0.981167  0.987583  0.978269  0.986323  0.962156  0.991868   
121622  0.932862  0.981167  0.987583  0.978269  0.986323  0.962156  0.991868   
121623  0.932862  0.981167  0.987583  0.978269  0.986323  0.962156  0.991868   

100%|██████████| 291456/291456 [00:00<00:00, 2436941.34it/s]


--Complete-- 5
num_node 121624
node_feat              0_x       1_x       2_x       3_x       4_x       5_x       6_x  \
0       1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
1       1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
2       1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
3       1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
4       1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
...          ...       ...       ...       ...       ...       ...       ...   
121619  0.932862  0.981167  0.987583  0.978269  0.986323  0.962156  0.991868   
121620  0.932862  0.981167  0.987583  0.978269  0.986323  0.962156  0.991868   
121621  0.932862  0.981167  0.987583  0.978269  0.986323  0.962156  0.991868   
121622  0.932862  0.981167  0.987583  0.978269  0.986323  0.962156  0.991868   
121623  0.932862  0.981167  0.987583  0.978269  0.986323  0.962156  0.991868   

100%|██████████| 485760/485760 [00:00<00:00, 2447104.70it/s]


--Complete-- 5
num_node 121624
node_feat              0_x       1_x       2_x       3_x       4_x       5_x       6_x  \
0       1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
1       1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
2       1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
3       1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
4       1.000000  0.897662  0.969539  0.977718  0.895714  0.955645  0.946404   
...          ...       ...       ...       ...       ...       ...       ...   
121619  0.932862  0.981167  0.987583  0.978269  0.986323  0.962156  0.991868   
121620  0.932862  0.981167  0.987583  0.978269  0.986323  0.962156  0.991868   
121621  0.932862  0.981167  0.987583  0.978269  0.986323  0.962156  0.991868   
121622  0.932862  0.981167  0.987583  0.978269  0.986323  0.962156  0.991868   
121623  0.932862  0.981167  0.987583  0.978269  0.986323  0.962156  0.991868   

100%|██████████| 681352/681352 [00:00<00:00, 2670431.91it/s]


OSError: 102772280 requested and 0 written